In [1]:
# 원하는 숫자 생성
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

# 하이퍼 파라미터들을 설정
total_epoch = 100
batch_size = 100
n_hidden = 256
n_input = 28 * 28
n_noise = 128
n_class = 10

# 신경망 모델 구성
X = tf.placeholder(tf.float32, [None, n_input])
Y = tf.placeholder(tf.float32, [None, n_class])
Z = tf.placeholder(tf.float32, [None, n_noise])

# 기존에는 각 신경망의 변수들을 따로따로 학습해야했지만 
# tf.layers를 사용하면 변수를 선언하지않고 tf.variable_scope를 이용해 스코프를 지정해줄 수 있음
# tf.concat 함수를 이용해 noise 값에 labels 정보를 간단하게 추가
# tf.layers.dense 함수를 이용해 은닉층을 만들고, 마찬가지로 진짜 이미지와 같은 크기의 값을 만드는 출력층(output)도 구성
def generator(noise, labels):
    with tf.variable_scope('generator'):
        # noise 값에 labels 정보를 추가합니다.
        inputs = tf.concat([noise, labels], 1)

        # TensorFlow 에서 제공하는 유틸리티 함수를 이용해 신경망을 매우 간단하게 구성할 수 있습니다.
        hidden = tf.layers.dense(inputs, n_hidden,
                                 activation=tf.nn.relu)
        output = tf.layers.dense(hidden, n_input,
                                 activation=tf.nn.sigmoid)

    return output

def discriminator(inputs, labels, reuse=None):
    with tf.variable_scope('discriminator') as scope:
        # 노이즈에서 생성한 이미지와 실제 이미지를 판별하는 모델의 변수를 동일하게 하기 위해,
        # 이전에 사용되었던 변수를 재사용하도록 합니다.
        if reuse:
            scope.reuse_variables()
        inputs = tf.concat([inputs, labels], 1)
        hidden = tf.layers.dense(inputs, n_hidden, activation=tf.nn.relu)        
        # 손실값 계산에 sigmoid_cross_entropy_with_logits 함수를 사용하기 위해 None을 사용
        output = tf.layers.dense(hidden, 1, activation=None)
    return output

# 노이즈를 균등분포로 생성하도록 작성
def get_noise(batch_size, n_noise):
    return np.random.uniform(-1., 1., size=[batch_size, n_noise])

# 생성자를 구성한 뒤 진짜 이미지 데이터와 생성자가 만든 이미지 데이터를 이용하는 구분자를 하나씩 만들어줌
# 생성자에는 레이블 정보를 추가하여 추후 레이블 정보에 해당하는 이미지를 생성할 수 있도록 유도
# 가짜 이미지 구분자를 만들 떄는 진짜 이미지 구분자에서 사용한 변수들을 재사용하도록 reuse 옵션을 True로 설정
G =generator(Z, Y)
D_real = discriminator(X, Y)
D_gene = discriminator(G, Y, True)

# loss_D_real은 D_real의 결괏값과 D_real의 크기만큼 1로 채운 값들을 비교(ones_like 함수)
loss_D_real = tf.reduce_mean(
                tf.nn.sigmoid_cross_entropy_with_logits(
                logits = D_real, labels=tf.ones_like(D_real)))

# loss_D_gene은 D_gene의 결괏값과 D_gene의 크기만큼 0으로 채운 값들을 비교하도록 함(zeros_like 함수)
loss_D_gene = tf.reduce_mean(
                tf.nn.sigmoid_cross_entropy_with_logits(
                logits=D_gene, labels=tf.zeros_like(D_gene)))

# loss_D 값을 최소화 시키면 구분자(경찰)를 학습시킬 수 있음
# D_real은 1에 가까워야하고 (실제 이미지는 진짜라고 판별), D_gene는 0에 가까워야 함(생성한 이미지는 가짜라고 판별)
loss_D = loss_D_real + loss_D_gene

# loss_G 를 구함 -> 생성자(위조지폐범)를 학습시키기 위한 손실값 D_gene를 1에 가깝게 만드는 값을 손실값을 취하도록 함
loss_G = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_gene, labels=tf.ones_like(D_gene)))
# tf.get_collection 함수를 이용해 discriminator와 generator 스코프에서 사용된 변수들을 가져온 뒤, 
# 이변수들을 최적화에 사용할 각각으 ㅣ손실 함수와 함께 최적화 함수에 넣어 학습 모델 구성을 마무리
vars_D = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope = 'discriminator')
vars_G = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope = 'generator')

train_D = tf.train.AdamOptimizer().minimize(loss_D, var_list=vars_D)
train_G = tf.train.AdamOptimizer().minimize(loss_G, var_list=vars_G)

# 학습 시작
sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples / batch_size)
loss_val_D, loss_val_G = 0, 0

for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size, n_noise)
        
        _, loss_val_D = sess.run([train_D, loss_D], feed_dict={X: batch_xs, Y: batch_ys, Z: noise})
        _, loss_val_G = sess.run([train_G, loss_G], feed_dict={Y: batch_ys, Z: noise})
    
    print('Epoch:', '%04d' % epoch,
          'D_loss: {:.4}'.format(loss_val_D),
          'G_loss: {:.4}'.format(loss_val_G))
    
    # 학습 결과를 확인
    # 노이즈를 만들고 이것을 생성자 G에 넣어 결괏값을 만든 뒤 
    # 이결과값들을 28*28 크기의 가짜 이미지로 만들어 samples 폴더에 저장하도록함
    if epoch == 0 or (epoch + 1) % 10 == 0:
        sample_size = 10
        noise = get_noise(sample_size, n_noise)
        samples = sess.run(G, feed_dict = {Y: mnist.test.labels[:sample_size], Z: noise})
        fig, ax = plt.subplots(2, sample_size, figsize=(sample_size, 2))
        
        for i in range(sample_size):
            ax[0][i].set_axis_off()
            ax[1][i].set_axis_off()
            
            ax[0][i].imshow(np.reshape(mnist.test.images[i], (28, 28)))
            ax[1][i].imshow(np.reshape(samples[i], (28, 28)))
        
        plt.savefig('samples2/{}.png'.format(str(epoch).zfill(3)),
                   bbox_inches='tight')
        plt.close(fig)
        
print('최적화 완료')

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Epoch: 0000 D_loss: 0.003359 G_loss: 7.903
Epoch: 0001 D_loss: 0.01768 G_loss: 7.374
Epoch: 0002 D_loss: 0.01934 G_loss: 7.3
Epoch: 0003 D_loss: 0.01058 G_loss: 7.417
Epoch: 0004 D_loss: 0.007963 G_loss: 8.626
Epoch: 0005 D_loss: 0.03832 G_loss: 7.559
Epoch: 0006 D_loss: 0.03309 G_loss: 7.197